In [1]:
import sys
sys.path.insert(0, '../funzioni')
from funzioni_ingestion import *

def check_and_scrape_company_pages(df_company_overview, df, file_path):
    old_links = list(df_company_overview['job_link'].unique())
    links = list(df['job_link'].unique())
    new_links = list(set(links) - set(old_links))
    print('Pagine su cui fare scraping', len(new_links))
    df_scraping_company=pd.DataFrame()
    n=1
    for i in range(0, len(new_links), n):
        df_fitt = scraping_company_page(new_links, end_n_page = i+n, start_n_page=i)
        df_scraping_company = df_scraping_company.append(df_fitt).reset_index(drop=True)
        df_company_overview.append(df_scraping_company).reset_index(drop=True).fillna('Non_disponibile').to_csv('../data/company_overview.csv', index=False)
        print('Lista parziale', len(df_scraping_company),'Lista totale', len(pd.read_csv('../data/company_overview.csv')))
    df_final = df.merge(df_scraping_company, how='left', on ='job_link').drop_duplicates().reset_index(drop=True)
    df_final.to_csv(file_path, index=False)

def run_scraping(url=None, file_esclusi=[], includi_solo=[]):
    import time
    from datetime import date
    path_overview = '../data/company_overview.csv'
    path = '../data'
    today = date.today()

    data = today.strftime("%Y%m%d")
    data_file = today.strftime("%Y-%m-%d")
    
    if url == None:
        print('Ricerco solo schede di valutazione')
        df_company_overview = pd.read_csv(path_overview)
        file_list = os.listdir(path)
        file_da_escludere = ['data_backup', 'company_overview.csv', 'jobs']
        file_list = list(set(file_list) - set(file_da_escludere) - set(file_esclusi))
        if len(includi_solo)>0:
            file_list = includi_solo
        for e in file_list:
            df_partial_company=pd.DataFrame()
            file_path = path + '/' + e
            df = pd.read_csv(file_path).drop_duplicates().reset_index(drop=True)
            check_and_scrape_company_pages(df_company_overview, df, file_path)
            print('Ho completato il file nel path', file_path)
    else:
        print('Faccio lo scraping del link che mi hai chiesto e poi ricerco le schede di valutazione')
        df_job_scraping = scraping_job_page(url)
        df_company_overview = pd.read_csv(path_overview)
        df_job_scraping['scraping_date'] = data_file
        file_path=path +"/jobs/scraping_job_" + data + '.csv'
        df_job_scraping.to_csv(file_path, index=False)
        check_and_scrape_company_pages(df_company_overview, df_job_scraping, file_path)
        
def parsing_data():
    import pandas as pd
    import os
    path = '../data'
    
    float_columns = [
            'company_rating','oppurtunita_carriera', 'stipendio_e_benefit',
           'cultura_e_valori', 'dirigenti_senior',
           'equilibrio_lavoro_vita_privata']
    
    string_columns = [
        'company', 'job_title', 'location', 'job_age',
           'job_link', 'sede_centrale', 'dimensioni', 'tipo', 'settore', 'segmento', 'entrate']
    
    integer_columns = ['fondata_nel']
    
    dfaaa = pd.read_csv('../data/company_overview.csv')
    
    file_list = os.listdir(path)
    file_da_escludere = ['data_backup', 'company_overview.csv', 'jobs']
    
    file_list = list(set(file_list) - set(file_da_escludere))
    for e in file_list:
        file_path = path + '/' + e
        print(file_path)
        df = pd.read_csv(file_path).drop_duplicates().reset_index(drop=True)
        df = df[['company', 'job_title', 'location', 'company_rating', 'job_age', 'job_link']]
        df = df.merge(dfaaa, how='left', on='job_link')
        #file_path[-12:-4]
        a = pd.to_datetime(file_path[-12:-4], format='%Y%m%d').strftime('%Y-%m-%d')
        
        df['scraping_date'] = a
        try:
            df['company_rating'] = df['company_rating'].str.extract(r'>\s*([0-9.]+)\s*<')
        except:
            pass
        for e in float_columns:
            df[e] = df[e].replace('Non_disponibile',0).astype(float)
        
        for e in string_columns:
            df[e] = df[e].astype(str)
            
        for e in integer_columns:
            #mask = np.isnan(df[e].replace('Non_disponibile',0).astype(float))
            #df.loc[mask, e] = 0
            df[e] = df[e].fillna(0).replace('Non_disponibile',0).astype(float).astype(int)
            
        df.to_csv(file_path, index=False)
    
        #for i in range(1, 13):
        #    a = dfc.columns[i]
        #    b = dfc.columns[i+12]
        #    mask = dfc[a] == 'Non_disponibile'
        #    dfc.loc[mask, a] = dfc.loc[mask, b]
        #

C:\Users\Casulippo\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
#run_scraping(url='https://www.glassdoor.it/Lavoro/firenze-lavori-SRCH_IL.0,7_IC2747825.htm')
run_scraping(url=None, file_esclusi=[], includi_solo=[''])

In [13]:
parsing_data()

../data/scraping_all_20230203.csv
../data/scraping_all_20230221.csv
../data/scraping_all_20230215.csv
../data/scraping_all_20230207.csv
../data/scraping_all_20230204.csv
../data/scraping_all_20230205.csv
../data/scraping_all_20230209.csv
../data/scraping_all_20230202.csv
../data/scraping_all_20230220.csv
../data/scraping_all_20230228.csv
../data/scraping_all_20230226.csv
../data/scraping_all_20230201.csv
../data/scraping_all_20230227.csv
../data/scraping_all_20230210.csv
../data/scraping_all_20230206.csv
../data/scraping_all_20230218.csv
../data/scraping_all_20230222.csv
../data/scraping_all_20230225.csv
../data/scraping_all_20230131.csv
../data/scraping_all_20230208.csv
../data/scraping_all_20230224.csv
../data/scraping_all_20230212.csv
../data/scraping_all_20230219.csv
../data/scraping_all_20230214.csv
../data/scraping_all_20230217.csv
../data/scraping_all_20230213.csv
../data/scraping_all_20230211.csv
../data/scraping_all_20230216.csv


In [14]:
import pandas as pd
import os
df_append = pd.DataFrame()
path = '../data'
file_list = os.listdir(path)
file_da_escludere = ['data_backup', 'company_overview.csv', 'jobs']
file_list = list(set(file_list) - set(file_da_escludere))
for e in file_list:
    file_path = path + '/' + e
    df = pd.read_csv(file_path).drop_duplicates().reset_index(drop=True)
    df_append = df_append.append(df).drop_duplicates().reset_index(drop=True)
df_1 = df_append.groupby(['scraping_date'])[['job_link']].count().reset_index().rename(columns={'job_link':'total_rows'})
df_2 = df_append[df_append['oppurtunita_carriera'].isna()].\
groupby(['scraping_date'])[['job_link']].count().reset_index().rename(columns={'job_link':'ko_schedevalutazione'})
df_eval = df_1.merge(df_2, on = 'scraping_date')
df_eval['ratio'] = df_eval['ko_schedevalutazione']/df_eval['total_rows']*100
df_eval

,scraping_date,total_rows,ko_schedevalutazione,ratio
0,2023-01-31,450,97,21.555556
1,2023-02-01,253,171,67.588933
2,2023-02-02,462,331,71.645022
3,2023-02-03,465,345,74.193548
4,2023-02-04,450,227,50.444444
5,2023-02-05,459,366,79.738562
6,2023-02-06,255,163,63.921569
7,2023-02-07,450,443,98.444444
8,2023-02-08,450,445,98.888889
9,2023-02-09,450,402,89.333333
